In [ ]:
import pandas as pd

def Room(house):
    i = house.find('室')
    j = house.find('厅')
    room = house[0:i]
    if room != '--':
        return int(room)
    else:
        return -1

def LivingRoom(house):
    i = house.find('室')
    j = house.find('厅')
    living = house[i + 1:j]
    if living != '--':
        return int(living)
    else:
        return -1

def square(mianji):
    s = mianji[0:mianji.find('平')]
    if s == '0':
        return -1
    elif s != '--':
        return int(s)
    else:
        return -1

def CJdanjia(cjdanjia):
    s = cjdanjia[0:cjdanjia.find('元')]
    if int(s) <= 9999:
        return -1
    else:
        return int(s)

def Comp(line, xiaoqu, mendian,attribute):
    room = line['room']
    living = line['living']
    mianji = line['mianji']
    danjia = line['danjia']
    xiaoqu_name = line['xiaoqu']
    mendian_name = line['mendian']

    xiaoqu_set = xiaoqu.get_group(xiaoqu_name).reset_index(drop=True)
    mendian_set = mendian.get_group(mendian_name).reset_index(drop=True)

    xiaoqu_mean = round(xiaoqu_set[attribute].mean())
    mendian_mean = round(mendian_set[attribute].mean())

    xiaoqu_set = xiaoqu_set.drop(xiaoqu_set[xiaoqu_set[attribute] == -1].index).reset_index(drop=True)
    mendian_set = mendian_set.drop(mendian_set[mendian_set[attribute] == -1].index).reset_index(drop=True)

    distance_xiaoqu = abs((xiaoqu_set['room'] - room) * 20) + abs((xiaoqu_set['living'] - living) * 20) +  \
               abs(xiaoqu_set['mianji'] - mianji) + abs(xiaoqu_set['danjia'] - danjia)

    distance_mendian = abs((mendian_set['room'] - room) * 20) + abs((mendian_set['living'] - living) * 20) +  \
               abs(mendian_set['mianji'] - mianji) + abs(mendian_set['danjia'] - danjia)

    if distance_xiaoqu.empty and distance_mendian.empty:
        if xiaoqu_set.__len__() < 2:
            return mendian_mean
        else:
            return xiaoqu_mean
    elif ~distance_xiaoqu.empty and ~distance_mendian.empty:
        if distance_xiaoqu.min()<distance_mendian.min():
            index = distance_xiaoqu.idxmin()
            return xiaoqu_set.iloc[index][attribute]
        else:
            index = distance_mendian.idxmin()
            return mendian_set.iloc[index][attribute]
    elif ~distance_xiaoqu.empty and distance_mendian.empty:
        index = distance_mendian.idxmin()
        return mendian_set.iloc[index][attribute]
    else:
        index = distance_xiaoqu.idxmin()
        return xiaoqu_set.iloc[index][attribute]


data = pd.read_csv('data/lianjia1.csv',encoding='gbk')
data['xiaoqu'],data['huxing'],data['mianji']=data['cjxiaoqu'].str.split(' ').str
data['room'] = data['huxing'].map(Room).fillna(-1)
data['living'] = data['huxing'].map(LivingRoom).fillna(-1)
data['mianji'] = data['mianji'].map(square).fillna(-1)
data['danjia'] = data['cjdanjia'].map(CJdanjia).fillna(-1)
xiaoqu_mean = data.groupby('xiaoqu')['mianji'].mean().astype(int)
mendian_mean = data.groupby('mendian')['mianji'].mean().astype(int)
mendian = data.groupby('mendian')
xiaoqu = data.groupby('xiaoqu')

for i in range(0,data.__len__()):
#i = 243
    line = data.iloc[i]

    if line['room'] == -1:
        line['room'] = Comp(line,xiaoqu,mendian,'room')

    if line['living'] == -1:
        line['living'] = Comp(line, xiaoqu, mendian,'living')

    if line['mianji'] <= 0:
        line['mianji'] = Comp(line, xiaoqu, mendian,'mianji')

    if line['danjia'] <= 0:
        line['danjia'] = Comp(line, xiaoqu, mendian,'danjia')
        line['cjzongjia'] = line['mianji'] * line['danjia'] / 10000

    data.iloc[i] = line

data['room'] = data['room'].astype(str)
data['living'] = data['living'].astype(str)
data['mianji'] = data['mianji'].astype(str)
data['danjia'] = data['danjia'].astype(str)
data['danjia'] = data['danjia']+'元/平'

data['new'] = data['xiaoqu']+' '+data['room']+'室'+data['living']+'厅'+' '+data['mianji']+'平'

data.drop(['huxing','room','mianji','cjdanjia','living','cjxiaoqu'], axis=1)
data.to_csv('./generated_data/lianjia.csv',encoding='gbk',columns=['cjtaoshu','mendian','cjzongjia','zhiwei','haoping','danjia',
                                                  'new','xingming','cjzhouqi','biaoqian','cjlouceng','cjshijian',
                                                  'congyenianxian','bankuai'])



In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
import numpy as np
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
print('加载数据集')
data = pd.read_csv('lianjia.csv', index_col=0,encoding='gbk')
print('属性类别数:', len(data.columns))
print('总行数:', len(data))
print('示例数据:')
data.head(5)

In [ ]:
print('提取每一列属性以及名称')
num_fields = data.select_dtypes(include=np.number).columns.values
nom_fields = data.select_dtypes(exclude=np.number).columns.values
print('标称属性:', nom_fields)
print('数值属性:', num_fields)
print(data.shape," ",nom_fields.shape," ",num_fields.shape)

In [ ]:
print("统计每一个属性的个数")
print('\nmendian(门店):')
a=data['mendian'].value_counts()
print(a)

print('\nzhiwei(职位):')
b=data['zhiwei'].value_counts()
print(b)

print('\nhaoping(好评):')
c=data['haoping'].value_counts()
print(c)

print('\ndanjia(成交单价):')
d=data['danjia'].value_counts()
print(d)

print('\nnew(成交单价):')
d1=data['new'].value_counts()
print(d1)

print('\nxingming(姓名):')
e=data['xingming'].value_counts()
print(e)

print('\nbiaoqian(标签):')
f=data['biaoqian'].value_counts()
print(f)

print('\ncjlouceng(成交楼层):')
g=data['cjlouceng'].value_counts()
print(g)

print('\ncjshijian(成交时间):')
h=data['cjshijian'].value_counts()
print(h)

print('\ncongyenianxian(从月年限):')
i=data['congyenianxian'].value_counts()
print(i)

print('\nbankuai(板块):')
j=data['bankuai'].value_counts()
print(j)

In [ ]:
data.describe()

In [ ]:
print('数据可视化：\n')
field = ['cjzongjia','cjzhouqi','danjia','cjtaoshu']
print('数据太多导致显示数据出现重叠看不清具体值，所以取前20个为例:')

for fie in field:
    print("{}前20个样例:".format(fie))
    data[fie].value_counts(sort=False).head(20).plot.barh()
    plt.show()

In [ ]:
print("画出不同店家的成交数量饼图")
label=[]
for key in data['mendian'].value_counts().index:
    label.append(key)

data['mendian'].value_counts().plot.pie(labels=label, 
                 autopct='%.2f', fontsize=10,figsize=(12, 12))

In [ ]:
print("画出不同从月年限的饼图")
label=[]
for key in data['congyenianxian'].value_counts().index:
    label.append(key)

data['congyenianxian'].value_counts().plot.pie(labels=label, 
                 autopct='%.2f', fontsize=15,figsize=(12, 12))

In [ ]:
print("画出不同板块（地区）的饼图")
label=[]
for key in data['bankuai'].value_counts().index:
    label.append(key)

data['bankuai'].value_counts().plot.pie(labels=label, 
                 autopct='%.2f', fontsize=15,figsize=(12, 12))

In [ ]:
print("画出不同标签的饼图")
label=[]
for key in data['biaoqian'].value_counts().index:
    label.append(key)

data['biaoqian'].value_counts().plot.pie(labels=label, 
                 autopct='%.2f', fontsize=15,figsize=(12, 12))